WEBSOCKET

In [ ]:
import asyncio
import datetime
import json
import pandas as pd
import numpy as np
import websockets
import pyarrow as pa
import pyarrow.parquet as pq
import logging

logging.basicConfig(format='%(asctime)s %(message)s', level=logging.DEBUG)

url = "wss://ws.okx.com:8443/ws/v5/public"
url_block = 'wss://wsaws.okx.com:8443/ws/v5/business'

df_=pd.DataFrame()
total_table = pd.DataFrame()

subs = dict(
    op='subscribe',
    args=(
        dict(channel='liquidation-orders', instType='SWAP'),
        dict(channel='trades', instId='BTC-USDT-SWAP'),
        dict(channel='books5', instId='BTC-USDT-SWAP'),
        dict(channel='open-interest', instId='BTC-USDT-SWAP'),
        dict(channel='trades', instId='ETH-USDT-SWAP'),
        dict(channel='books5', instId='ETH-USDT-SWAP'),
        dict(channel='open-interest', instId='ETH-USDT-SWAP'),
        dict(channel='trades', instId='AVAX-USDT-SWAP'),
        dict(channel='books5', instId='AVAX-USDT-SWAP'),
        dict(channel='open-interest', instId='AVAX-USDT-SWAP'),
        dict(channel='trades', instId='SOL-USDT-SWAP'),
        dict(channel='books5', instId='SOL-USDT-SWAP'),
        dict(channel='open-interest', instId='SOL-USDT-SWAP'),
        dict(channel='trades', instId='IOTA-USDT-SWAP'),
        dict(channel='books5', instId='IOTA-USDT-SWAP'),
        dict(channel='open-interest', instId='IOTA-USDT-SWAP'),
        dict(channel='trades', instId='LINK-USDT-SWAP'),
        dict(channel='books5', instId='LINK-USDT-SWAP'),
        dict(channel='open-interest', instId='LINK-USDT-SWAP'),
        dict(channel='trades', instId='SUI-USDT-SWAP'),
        dict(channel='books5', instId='SUI-USDT-SWAP'),
        dict(channel='open-interest', instId='SUI-USDT-SWAP'),
        dict(channel='trades', instId='INJ-USDT-SWAP'),
        dict(channel='books5', instId='INJ-USDT-SWAP'),
        dict(channel='open-interest', instId='INJ-USDT-SWAP'),            
    )
)

subs_block = dict(
    op='subscribe',
    args=(
        dict(channel='block-tickers', instId='BTC-USDT-SWAP'),
        dict(channel='block-tickers', instId='ETH-USDT-SWAP'),
        #dict(channel='block-tickers', instId='AVAX-USDT-SWAP'),
        #dict(channel='block-tickers', instId='SOL-USDT-SWAP'),
        #dict(channel='block-tickers', instId='IOTA-USDT-SWAP'),
        #dict(channel='block-tickers', instId='INJ-USDT-SWAP'),
        #dict(channel='block-tickers', instId='LINK-USDT-SWAP'),
        #dict(channel='block-tickers', instId='SUI-USDT-SWAP'),
    )
)


async def send_subscribe_msg(ws,subs,total_table):
    await ws.send(json.dumps(subs))
    #print(f'subs: {json.dumps(subs)}')
    async for msg in ws:
        msg = json.loads(msg)
        ev = msg.get('event')
        arg = msg.get('arg')
        data = msg.get('data')
        channel = arg.get('channel')
        if ev:
            i = msg.get('arg')
            print(f'****************** event {ev} = {i} ******************')

        elif channel == 'trades':
            instId=data[0].get('instId')
            tradeId=data[0].get('tradeId')
            px=data[0].get('px')
            sz=data[0].get('sz')
            side=data[0].get('side')
            ts=data[0].get('ts')
            count=data[0].get('count')
            lst = [channel,instId,tradeId,px,sz,side,ts,count]
            columns = ['channel','instId','tradeId','px','sz','side','ts','count']
            df_ = pd.DataFrame(lst,columns).T
        
        elif channel == 'books5':
            asks = data[0].get('asks')
            bids = data[0].get('bids')
            instId = data[0].get('instId')
            ts = data[0].get('ts')
            seqId = data[0].get('seqId')
            asks = list(np.array(asks).ravel())
            bids = list(np.array(bids).ravel())

            lst=[channel,instId,ts,seqId]
            lst.extend(asks)
            lst.extend(bids)
            columns = ['channel','instId','ts','seqId','ask_price1','ask_size1','na1','ask_counts1','ask_price2','ask_size2','na2','ask_counts2','ask_price3','ask_size3','na3','ask_counts3','ask_price4','ask_size4','na4','ask_counts4','ask_price5','ask_size5','na5','ask_counts5','bid_price1','bid_size1','nal1','bid_counts1','bid_price2','bid_size2','nal2','bid_counts2','bid_price3','bidk_size3','nal3','bid_counts3','bid_price4','bid_size4','nal4','bid_counts4','bid_price5','bid_size5','nal5','bid_counts5']

            df_ = pd.DataFrame(lst,columns).T

        
        elif channel == 'open-interest':
            instId = data[0].get('instId')
            instType = data[0].get('instType')
            oi = data[0].get('oi')
            oiCcy = data[0].get('oiCcy')
            ts = data[0].get('ts')
            lst = [channel,instId,instType,oi,oiCcy,ts]
            columns = ['channel','instId','instType','oi','oiCcy','ts']
            df_ = pd.DataFrame(lst,columns).T

        elif channel=='liquidation-orders':

            instType = data[0].get('instType')
            instId = data[0].get('instId')
            uly = data[0].get('uly')
            details = data[0].get('details')
            side = details.get('side')
            posSide = details.get('posSide')
            bkPx = details.get('bkPx')
            sz = details.get('sz')
            bkLoss = details.get('bkLoss')
            ccy = details.get('ccy')
            ts = details.get('ts')

            if instId =='BTC-USDT-SWAP'or instId == 'ETH-USDT-SWAP'or instId == 'AVAX-USDT-SWAP'or instId == 'SOL-USDT-SWAP'or instId == 'IOTA-USDT-SWAP'or instId == 'LINK-USDT-SWAP'or instId == 'SUI-USDT-SWAP'or instId =='INJ-USDT-SWAP':
                lst = [channel,instId, side, posSide, bkPx, sz, bkLoss, ccy, ts]
                columns = ['channel','instId', 'side', 'posSide', 'bkPx', 'sz', 'bkLoss', 'ccy', 'ts']
                df_ = pd.DataFrame(lst,columns).T
    
        elif channel=='block-tickers':
            instType = data[0].get("instType")
            instId = data[0].get("instId")
            volCcy24h = data[0].get("volCcy24h")
            vol24h = data[0].get("vol24h")
            ts = data[0].get("ts")
            lst = [channel,instId,instType,volCcy24h,vol24h,ts]
            columns = ['channel','instId','instType','volCcy24h','vol24h,','ts']
            df_ = pd.DataFrame(lst,columns).T
        if data :
            total_table= await table_concat(total_table, df_)
            print(total_table.shape)
        if total_table.shape[0]>500000:
            total_table = await write_table_parquet(total_table)
            
async def table_concat(total_table, df_):
    return pd.concat([total_table, df_], axis=0,ignore_index=True)

async def write_table_parquet(total_table):
    table = pa.Table.from_pandas(total_table)
    pq.write_table(table, f'train{datetime.datetime.now()}.parquet')
    return pd.DataFrame()

async def base_ws():
    try:
        async with websockets.connect(url, ping_interval=15,ping_timeout=60) as ws:
            await asyncio.create_task(send_subscribe_msg(ws,subs,total_table))
    except CancelledError as e:
        print('Already cancelled the client...')
    except ConnectionClosedError as e:
        print(e)
    await asyncio.sleep(1)
    await base_ws()
async def block_ws():
    try:
        async with websockets.connect(url_block, ping_interval=15,ping_timeout=60) as ws:
            await asyncio.create_task(send_subscribe_msg(ws,subs_block,total_table))
    except CancelledError as e:
        print('Already cancelled the client...')
    except ConnectionClosedError as e:
        print(e)
    await asyncio.sleep(1)
    await block_ws()
async def executer():
    start_time = datetime.datetime.now().isoformat()
    print('Connected '+ start_time)
    await asyncio.wait([
            asyncio.create_task(base_ws()),
            asyncio.create_task(block_ws()),
    ])
    end_time = datetime.datetime.now().isoformat()
    print('Disconnected '+ end_time)

if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    await executer()

Connected 2024-03-16T13:38:15.188931
****************** event subscribe = {'channel': 'block-tickers', 'instId': 'BTC-USDT-SWAP'} ******************
****************** event subscribe = {'channel': 'block-tickers', 'instId': 'ETH-USDT-SWAP'} ******************
(1, 6)
(2, 6)
****************** event subscribe = {'channel': 'liquidation-orders', 'instType': 'SWAP'} ******************
****************** event subscribe = {'channel': 'trades', 'instId': 'BTC-USDT-SWAP'} ******************
****************** event subscribe = {'channel': 'books5', 'instId': 'BTC-USDT-SWAP'} ******************
****************** event subscribe = {'channel': 'open-interest', 'instId': 'BTC-USDT-SWAP'} ******************
****************** event subscribe = {'channel': 'trades', 'instId': 'ETH-USDT-SWAP'} ******************
****************** event subscribe = {'channel': 'books5', 'instId': 'ETH-USDT-SWAP'} ******************
****************** event subscribe = {'channel': 'open-interest', 'instId': 'ETH-US

trades
'channel','instId','tradeId','ts',     'px','sz','side','count'

books5
 'channel','instId','ts','seqId',         
                         ask_price{i}, ask_size{i}, ask_counts{i}, bid_price{i}, bid_size{i}, bid_counts{i},

open-interest
'channel','instId','instType','ts',          'oi','oiCcy'

liquidation-orders
'channel','instId', 'ts',               'side', 'posSide', 'bkPx', 'sz', 'bkLoss', 'ccy'

'block-tickers
'channel','instId','instType','ts',             'volCcy24h','vol24h,'

FEATURES

def load_data():
    return pd.read_hdf('train.h5', 'data')
def make_feature():
    pass
def make_trade_features():
    pass
def make_book_features():
    pass
def make_book_features_v2():
    pass
def make_open_interest_features():
    pass
def make_liquidation_orders_features():
    pass